In [1]:
import pandas as pd
import numpy as np

In [2]:
training_data = pd.read_csv("input_data_train.csv")
pred_data = pd.read_csv("input_data_pred.csv")

In [3]:
training_data.describe()

,location,product,sa_quantity,temp_mean,temp_max,temp_min,sunshine_quant,price
count,1.011060e+07,1.011060e+07,1.011060e+07,9.706176e+06,9.706176e+06,9.706176e+06,9.706176e+06,8.827168e+06
mean,2.419800e+03,1.850000e+02,8.604586e+00,8.665875e+00,1.173502e+01,5.596728e+00,3.629658e+02,2.875698e+00
std,2.155645e+03,1.065207e+02,1.303595e+01,5.113955e+00,5.717526e+00,4.884786e+00,1.812255e+02,1.911442e+00
min,5.500000e+01,1.000000e+00,0.000000e+00,-6.545000e+00,-2.930000e+00,-1.091000e+01,0.000000e+00,0.000000e+00
25%,4.370000e+02,9.300000e+01,2.000000e+00,4.960000e+00,7.820000e+00,1.970000e+00,2.250000e+02,1.600000e+00
50%,2.166500e+03,1.850000e+02,5.000000e+00,8.510000e+00,1.142000e+01,5.560000e+00,3.380000e+02,2.483333e+00
75%,3.353000e+03,2.770000e+02,1.000000e+01,1.247000e+01,1.598000e+01,9.230000e+00,4.880000e+02,3.300000e+00
max,9.137000e+03,3.690000e+02,8.190000e+02,2.563000e+01,3.028000e+01,2.098000e+01,1.022000e+03,5.610000e+01


In [4]:
training_data.head()

,location,product,date,sa_quantity,temp_mean,temp_max,temp_min,sunshine_quant,event,price
0,1193,1,2011-12-31,5,10.635,11.19,10.08,56.0,New Year's Eve,1.48
1,1193,1,2012-01-01,0,8.845,10.76,6.93,213.0,New Year's Day,NaN
2,1193,1,2012-01-02,2,5.510,7.24,3.78,285.0,New Year's Day observed,1.50
3,1193,1,2012-01-03,2,8.915,11.61,6.22,205.0,2nd January (substitute day),1.50
4,1193,1,2012-01-04,0,7.380,8.87,5.89,224.0,NaN,NaN


In [5]:
from sklearn import preprocessing

# Given a data frame and the columns containing numerical continuous features,
# output the data frame with normalized values per column.
def normalize_features(data_df, columns):
    for feature in columns:
        continuous_data = data_df[feature]
        continuous_data_values = continuous_data.values.reshape(-1, 1)

        min_max_scaler = preprocessing.MinMaxScaler()
        continuous_data_values_scaled = min_max_scaler.fit_transform(continuous_data_values)

        data_df[feature] = continuous_data_values_scaled

def encode_categorical_features(data_df, columns):
    encoders = {}
    for feature in columns:
        le = preprocessing.LabelEncoder()
        le.fit(data_df[feature].values)
        data_df[feature] = le.transform(data_df[feature].values)
        encoders[feature] = le
    return encoders
        
        
# Performs preprocessing on the data for the training and evaluation sets.
def preprocess(df, continuous_feature_columns=[], categorical_feature_columns=[], filter_columns=[],
               label_column="sa_quantity", date_column="date"):
    # Remove price: NaN records from table
    df = df.dropna()

    # Normalize numerical features
    df_Y = df[label_column]
    normalize_features(df, continuous_feature_columns)
    
    # Convert date column to timestamp
    df[date_column] = pd.to_datetime(df[date_column]).values.astype(np.int64)
    
    # Convert event to categorical value.
    encoders = encode_categorical_features(df, categorical_feature_columns)
    
    # Remove label_column from training set.
    df_columns = list(df.columns.values)
    df_columns.remove(label_column)
    
    # Remove filter columns.
    for column in filter_columns:
        df_columns.remove(column)
    
    df_X = df[df_columns]
    
    return df_X, df_Y, encoders

# Normalize training data continuous features.
continuous_feature_columns = ["temp_mean", "temp_max", "temp_min", "sunshine_quant", "price"]
categorical_feature_columns = ["event", "location", "product"]
filter_columns = []

X, y, encoders = preprocess(training_data, continuous_feature_columns, categorical_feature_columns,  filter_columns)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

In [6]:
corr_df = X

In [7]:
corr_df["y"] = y

In [8]:
corr_df.corr()

,location,product,date,temp_mean,temp_max,temp_min,sunshine_quant,event,price,y
location,1.000000,-0.001518,0.000565,-0.033002,-0.026354,-0.037945,-0.023911,-0.001058,-0.003491,-0.057327
product,-0.001518,1.000000,-0.000500,-0.001192,-0.001108,-0.001187,-0.000008,0.000326,-0.020977,-0.088531
date,0.000565,-0.000500,1.000000,-0.082711,-0.075912,-0.083501,0.003459,0.013862,0.012180,-0.018273
temp_mean,-0.033002,-0.001192,-0.082711,1.000000,0.966079,0.952770,0.414993,0.083306,-0.010654,-0.022712
temp_max,-0.026354,-0.001108,-0.075912,0.966079,1.000000,0.842023,0.550041,0.110324,-0.010902,-0.024139
temp_min,-0.037945,-0.001187,-0.083501,0.952770,0.842023,1.000000,0.220024,0.044275,-0.009434,-0.019055
sunshine_quant,-0.023911,-0.000008,0.003459,0.414993,0.550041,0.220024,1.000000,0.204937,-0.005814,-0.010632
event,-0.001058,0.000326,0.013862,0.083306,0.110324,0.044275,0.204937,1.000000,-0.002353,-0.011244
price,-0.003491,-0.020977,0.012180,-0.010654,-0.010902,-0.009434,-0.005814,-0.002353,1.000000,-0.141183
y,-0.057327,-0.088531,-0.018273,-0.022712,-0.024139,-0.019055,-0.010632,-0.011244,-0.141183,1.000000


In [9]:
corr_df.to_csv("corr_matrix.csv")

In [47]:
def quadratic_fit(a, b):
    fit = np.polyfit(a, b, 2)
    b_ = np.polyval(fit, a)
    return np.mean(np.square(b - b_))

In [48]:
a = np.array([1, 2, 3])
b = np.array([1, 2, 3])

In [49]:
quadratic_fit(a,b)

9.367723249499515e-30

In [50]:
corr_df.corr(method=quadratic_fit)

,location,product,date,temp_mean,temp_max,temp_min,sunshine_quant,event,price,y
location,1.000000e+00,1.135088e+04,1.942167e+32,0.028892,0.031564,0.026919,0.031679,680.037765,0.009528,159.053906
product,1.135088e+04,1.000000e+00,1.942165e+32,0.028945,0.031602,0.026983,0.031741,680.038471,0.009494,158.293773
date,1.942167e+32,1.942165e+32,1.000000e+00,0.028554,0.031313,0.026515,0.031653,674.654890,0.009527,159.724617
temp_mean,2.889224e-02,2.894464e-02,2.855433e-02,1.000000,0.002106,0.002487,0.026076,675.266269,0.009528,159.613729
temp_max,3.156448e-02,3.160180e-02,3.131259e-02,0.002106,1.000000,0.007833,0.021966,671.597630,0.009528,159.574284
temp_min,2.691946e-02,2.698285e-02,2.651511e-02,0.002487,0.007833,1.000000,0.029935,678.699357,0.009528,159.685882
sunshine_quant,3.167853e-02,3.174147e-02,3.165252e-02,0.026076,0.021966,0.029935,1.000000,650.803156,0.009529,159.635006
event,6.800378e+02,6.800385e+02,6.746549e+02,675.266269,671.597630,678.699357,650.803156,1.000000,0.009529,159.746625
price,9.528435e-03,9.493716e-03,9.526990e-03,0.009528,0.009528,0.009528,0.009529,0.009529,1.000000,156.038611
y,1.590539e+02,1.582938e+02,1.597246e+02,159.613729,159.574284,159.685882,159.635006,159.746625,156.038611,1.000000


In [52]:
corr_df.corr(method="kendall")

,location,product,date,temp_mean,temp_max,temp_min,sunshine_quant,event,price,y
location,1.000000,-0.001020,0.000448,-0.021559,-0.017393,-0.023991,-0.012312,-0.000691,-0.000775,-0.043460
product,-0.001020,1.000000,-0.000315,-0.000680,-0.000615,-0.000704,0.000128,0.000208,0.024771,-0.066885
date,0.000448,-0.000315,1.000000,-0.046378,-0.043795,-0.049314,0.003988,0.012764,0.007999,-0.016367
temp_mean,-0.021559,-0.000680,-0.046378,1.000000,0.835564,0.813275,0.254422,0.062448,-0.006574,-0.013354
temp_max,-0.017393,-0.000615,-0.043795,0.835564,1.000000,0.648484,0.357186,0.077270,-0.006949,-0.016426
temp_min,-0.023991,-0.000704,-0.049314,0.813275,0.648484,1.000000,0.127130,0.040554,-0.005710,-0.008688
sunshine_quant,-0.012312,0.000128,0.003988,0.254422,0.357186,0.127130,1.000000,0.138795,-0.004419,-0.006185
event,-0.000691,0.000208,0.012764,0.062448,0.077270,0.040554,0.138795,1.000000,-0.001529,-0.009437
price,-0.000775,0.024771,0.007999,-0.006574,-0.006949,-0.005710,-0.004419,-0.001529,1.000000,-0.175034
y,-0.043460,-0.066885,-0.016367,-0.013354,-0.016426,-0.008688,-0.006185,-0.009437,-0.175034,1.000000
